In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-13 07:36:46.433980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 07:36:46.441217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 07:36:46.441661: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 07:36:46.442725: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001}
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=None)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_1_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 72)]         0         
_________________________________________________________________
flatten (Flatten)            (None, 7200)              0         
_________________________________________________________________
dropout (Dropout)            (None, 7200)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               3600500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_______________________________________________________

2022-07-13 07:36:53.537350: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


10/10 [==============================] - 1s 26ms/step - loss: 47.5340 - mean_squared_error: 47.5340 - rmse: 6.8945 - mean_absolute_error: 5.4304 - val_loss: 11.0411 - val_mean_squared_error: 11.0411 - val_rmse: 3.3228 - val_mean_absolute_error: 2.4727
Epoch 2/1000
10/10 [==============================] - 0s 10ms/step - loss: 15.4990 - mean_squared_error: 15.4990 - rmse: 3.9369 - mean_absolute_error: 3.0789 - val_loss: 9.6807 - val_mean_squared_error: 9.6807 - val_rmse: 3.1114 - val_mean_absolute_error: 2.4886
Epoch 3/1000
10/10 [==============================] - 0s 10ms/step - loss: 13.1315 - mean_squared_error: 13.1315 - rmse: 3.6237 - mean_absolute_error: 2.8985 - val_loss: 13.2342 - val_mean_squared_error: 13.2342 - val_rmse: 3.6379 - val_mean_absolute_error: 2.6558
Epoch 4/1000
10/10 [==============================] - 0s 10ms/step - loss: 12.8556 - mean_squared_error: 12.8556 - rmse: 3.5855 - mean_absolute_error: 2.8565 - val_loss: 11.0903 - val_mean_squared_error: 11.0903 - val_rm

In [11]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=1000)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_2_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_2_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 7200)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 500)               3600500   
_________________________________________________________________
dropout_41 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 500)               250500    
____________________________________________________

In [12]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_3_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dropout_80 (Dropout)         (None, 7200)              0         
_________________________________________________________________
dense_80 (Dense)             (None, 500)               3600500   
_________________________________________________________________
dropout_81 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 500)               250500    
____________________________________________________

In [13]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=350)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_4_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_4_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dropout_120 (Dropout)        (None, 7200)              0         
_________________________________________________________________
dense_120 (Dense)            (None, 500)               3600500   
_________________________________________________________________
dropout_121 (Dropout)        (None, 500)               0         
_________________________________________________________________
dense_121 (Dense)            (None, 500)               250500    
____________________________________________________

2022-07-13 07:49:14.977486: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


Epoch 23/1000
3/3 [==============================] - 0s 13ms/step - loss: 9.7357 - mean_squared_error: 9.7357 - rmse: 3.1202 - mean_absolute_error: 2.4361 - val_loss: 13.3140 - val_mean_squared_error: 13.3140 - val_rmse: 3.6488 - val_mean_absolute_error: 2.8429
Epoch 24/1000
3/3 [==============================] - 0s 13ms/step - loss: 10.7202 - mean_squared_error: 10.7202 - rmse: 3.2742 - mean_absolute_error: 2.5714 - val_loss: 12.1726 - val_mean_squared_error: 12.1726 - val_rmse: 3.4889 - val_mean_absolute_error: 2.7182
Epoch 25/1000
3/3 [==============================] - 0s 13ms/step - loss: 10.8311 - mean_squared_error: 10.8311 - rmse: 3.2911 - mean_absolute_error: 2.6493 - val_loss: 14.6925 - val_mean_squared_error: 14.6925 - val_rmse: 3.8331 - val_mean_absolute_error: 3.0025
Epoch 26/1000
3/3 [==============================] - 0s 13ms/step - loss: 10.3461 - mean_squared_error: 10.3461 - rmse: 3.2165 - mean_absolute_error: 2.5075 - val_loss: 12.6524 - val_mean_squared_error: 12.6524

In [14]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=175)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_5_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_5_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
flatten_40 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dropout_160 (Dropout)        (None, 7200)              0         
_________________________________________________________________
dense_160 (Dense)            (None, 500)               3600500   
_________________________________________________________________
dropout_161 (Dropout)        (None, 500)               0         
_________________________________________________________________
dense_161 (Dense)            (None, 500)               250500    
____________________________________________________

In [15]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=100)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_6_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_6_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
flatten_50 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dropout_200 (Dropout)        (None, 7200)              0         
_________________________________________________________________
dense_200 (Dense)            (None, 500)               3600500   
_________________________________________________________________
dropout_201 (Dropout)        (None, 500)               0         
_________________________________________________________________
dense_201 (Dense)            (None, 500)               250500    
____________________________________________________

2022-07-13 07:51:19.965490: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


1/1 [==============================] - 0s 19ms/step - loss: 9.7934 - mean_squared_error: 9.7934 - rmse: 3.1294 - mean_absolute_error: 2.5324 - val_loss: 5.5904 - val_mean_squared_error: 5.5904 - val_rmse: 2.3644 - val_mean_absolute_error: 1.7768
Epoch 160/1000
1/1 [==============================] - 0s 18ms/step - loss: 10.3014 - mean_squared_error: 10.3014 - rmse: 3.2096 - mean_absolute_error: 2.6751 - val_loss: 5.9081 - val_mean_squared_error: 5.9081 - val_rmse: 2.4307 - val_mean_absolute_error: 1.7731
Epoch 161/1000
1/1 [==============================] - 0s 18ms/step - loss: 8.0609 - mean_squared_error: 8.0609 - rmse: 2.8392 - mean_absolute_error: 2.2993 - val_loss: 5.9192 - val_mean_squared_error: 5.9192 - val_rmse: 2.4329 - val_mean_absolute_error: 1.7742
Epoch 162/1000
1/1 [==============================] - 0s 18ms/step - loss: 10.8031 - mean_squared_error: 10.8031 - rmse: 3.2868 - mean_absolute_error: 2.6131 - val_loss: 5.6635 - val_mean_squared_error: 5.6635 - val_rmse: 2.3798 -

/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  ret =

Model: "model_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
flatten_55 (Flatten)         (None, 7200)              0         
_________________________________________________________________
dropout_220 (Dropout)        (None, 7200)              0         
_________________________________________________________________
dense_220 (Dense)            (None, 500)               3600500   
_________________________________________________________________
dropout_221 (Dropout)        (None, 500)               0         
_________________________________________________________________
dense_221 (Dense)            (None, 500)               250500    
_________________________________________________________________
dropout_222 (Dropout)        (None, 500)               0  

OSError: Unable to open file (unable to open file: name = '/tmp/tmpc3_c8i2n.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)